<a href="https://colab.research.google.com/github/Xemgomez/ElaineGomez_DTSC3020.020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [ ]:
 #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [ ]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [3]:
# Q1 — Write your answer here


import re
import unicodedata
from typing import List

import requests
import pandas as pd


URL = "https://www.iban.com/country-codes"
OUTFILE = "data_q1.csv"


# --- Helpers ---------------------------------------------------------------

CONTROL_CHARS_RE = re.compile(r"[\x00-\x1F\x7F]")  # control chars incl. \n,\r,\t
MULTISPACE_RE = re.compile(r"\s+")

def _clean_text(val: str) -> str:
    """Unicode-normalize, drop control chars/zero-widths, collapse spaces."""
    if pd.isna(val):
        return ""
    s = str(val)
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u200b", "").replace("\ufeff", "")  # zero-width/BOM
    s = CONTROL_CHARS_RE.sub(" ", s)
    s = MULTISPACE_RE.sub(" ", s).strip()
    return s


# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    # Use pandas to parse all HTML tables
    tables: List[pd.DataFrame] = pd.read_html(html, flavor="lxml")
    if not tables:
        raise ValueError("No tables found on the page.")

    # Pick the first table with >= 3 columns
    for df in tables:
        if df.shape[1] >= 3:
            # Flatten headers (handles MultiIndex or messy headers)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = [
                    " ".join([_clean_text(x) for x in tup if pd.notna(x)]).strip()
                    for tup in df.columns.to_list()
                ]
            else:
                df.columns = [_clean_text(c) for c in df.columns]
            return df

    raise ValueError("No table with >= 3 columns found.")


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    # Try to detect canonical columns by name (case/spacing tolerant)
    colmap = {}
    for c in df.columns:
        lc = c.lower()
        if "country" in lc:
            colmap[c] = "country"
        elif "alpha-2" in lc or "alpha 2" in lc:
            colmap[c] = "alpha_2"
        elif "alpha-3" in lc or "alpha 3" in lc:
            colmap[c] = "alpha_3"
        elif "numeric" in lc:
            colmap[c] = "numeric"

    df = df.rename(columns=colmap)
    # Keep only the needed columns if present
    needed = ["country", "alpha_2", "alpha_3", "numeric"]
    present = [c for c in needed if c in df.columns]
    if len(present) < 4:
        raise ValueError(f"Expected columns not found. Got: {df.columns.tolist()}")

    df = df[present].copy()

    # Clean text columns
    for col in ["country", "alpha_2", "alpha_3"]:
        df[col] = df[col].astype(str).map(_clean_text)

    # Uppercase alpha codes
    df["alpha_2"] = df["alpha_2"].str.upper()
    df["alpha_3"] = df["alpha_3"].str.upper()

    # Clean numeric: keep digits only, then cast to nullable Int64
    df["numeric"] = (
        df["numeric"]
        .astype(str)
        .map(lambda s: re.sub(r"\D", "", s))
        .replace({"": pd.NA})
    )
    df["numeric"] = pd.to_numeric(df["numeric"], errors="coerce").astype("Int64")

    # Drop invalids: require non-empty country; alpha_2 len==2; alpha_3 len==3; numeric not NA
    df = df[
        (df["country"].str.len() > 0)
        & (df["alpha_2"].str.len() == 2)
        & (df["alpha_3"].str.len() == 3)
        & (df["numeric"].notna())
    ].copy()

    # Final whitespace polish
    for col in ["country", "alpha_2", "alpha_3"]:
        df[col] = df[col].str.strip()

    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    if "numeric" not in df.columns:
        raise ValueError("Column 'numeric' not found for sorting.")
    out = df.sort_values("numeric", ascending=False, na_position="last").head(top)
    return out.reset_index(drop=True)


# --- Main script -----------------------------------------------------------

def main():
    # Fetch HTML with a reasonable UA and encoding handling
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }
    r = requests.get(URL, headers=headers, timeout=30)
    if not r.encoding:
        r.encoding = r.apparent_encoding or "utf-8"
    html = r.text

    # Pipeline: read -> clean -> sort -> top 15
    raw_df = q1_read_table(html)
    clean_df = q1_clean(raw_df)
    top_df = q1_sort_top(clean_df, top=15)

    # Write CSV (UTF-8 with BOM for Excel friendliness)
    top_df.to_csv(OUTFILE, index=False, encoding="utf-8-sig")
    print(f"Wrote {len(top_df)} rows to {OUTFILE}")


if __name__ == "__main__":
    main()

Wrote 15 rows to data_q1.csv


/tmp/ipython-input-2802630245.py:39: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables: List[pd.DataFrame] = pd.read_html(html, flavor="lxml")


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [5]:
# Q2 — Write your answer here


    #!/usr/bin/env python3
# -*- coding: utf-8 -*-

import html
import re
import time
import unicodedata
from typing import List, Dict, Any

import requests
from bs4 import BeautifulSoup
import pandas as pd


HN_URL = "https://news.ycombinator.com/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; HN-Q2-Scraper/1.1; +https://news.ycombinator.com/)"
}


def _fetch(url: str, retries: int = 3, backoff: float = 1.5) -> str:
    last = None
    for i in range(retries):
        try:
            resp = requests.get(url, headers=HEADERS, timeout=15)
            resp.raise_for_status()
            resp.encoding = resp.apparent_encoding or "utf-8"
            return resp.text
        except Exception as e:
            last = e
            time.sleep(backoff ** (i + 1))
    raise RuntimeError(f"Failed to fetch {url}: {last}")


def _clean_text(s: str) -> str:
    """Normalize & remove undesirable/control chars."""
    if not s:
        return ""
    s = html.unescape(s)
    s = s.replace("\xa0", " ")
    s = unicodedata.normalize("NFKC", s)
    s = "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")
    return s.strip()


def _to_int_or_zero(text: str) -> int:
    if not text:
        return 0
    m = re.search(r"\d+", text.replace(",", ""))
    return int(m.group()) if m else 0


# --- Q2 Skeleton (filled) ---
def q2_parse_items(html_text: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html_text, "html.parser")
    rows: List[Dict[str, Any]] = []

    for athing in soup.select("tr.athing"):
        # Title & link
        title_a = athing.select_one("span.titleline > a")
        title = _clean_text(title_a.get_text(strip=True)) if title_a else ""
        link = title_a["href"].strip() if title_a and title_a.has_attr("href") else ""

        # Rank like "1."
        rank_txt = _clean_text((athing.select_one("span.rank") or {}).get_text() if athing else "")
        # subtext row (points, user, age, comments) is the next <tr>
        subtext = None
        nxt = athing.find_next_sibling("tr")
        if nxt:
            subtext = nxt.select_one("td.subtext")

        # Points like "123 points"
        points_txt = ""
        if subtext:
            score = subtext.select_one("span.score")
            points_txt = _clean_text(score.get_text()) if score else ""

        # Comments and user
        comments_txt = ""
        user_txt = ""
        if subtext:
            user_a = subtext.select_one("a.hnuser")
            user_txt = _clean_text(user_a.get_text()) if user_a else ""
            anchors = subtext.find_all("a")
            if anchors:
                last_a_txt = _clean_text(anchors[-1].get_text())
                if "comment" in last_a_txt.lower():
                    comments_txt = last_a_txt

        rows.append(
            {
                "rank": rank_txt,
                "title": title,
                "link": link,
                "points": points_txt,
                "comments": comments_txt,
                "user": user_txt,
            }
        )

    return pd.DataFrame(rows)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    out = df.copy()

    # Ensure expected columns exist
    for col in ["rank", "title", "link", "points", "comments", "user"]:
        if col not in out.columns:
            out[col] = ""

    # Clean/normalize text fields
    for col in ["title", "link", "user"]:
        out[col] = out[col].astype(str).map(_clean_text).fillna("")

    # Cast numerics (non-digits -> 0)
    out["rank"] = out["rank"].astype(str).map(_to_int_or_zero)
    out["points"] = out["points"].astype(str).map(_to_int_or_zero)
    out["comments"] = out["comments"].astype(str).map(_to_int_or_zero)

    # Make sure dtypes are plain ints
    out["rank"] = out["rank"].astype(int)
    out["points"] = out["points"].astype(int)
    out["comments"] = out["comments"].astype(int)

    return out


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return (
        df.sort_values(["points", "rank"], ascending=[False, True])
          .head(top)
          .reset_index(drop=True)
    )


# --- Runner ---
def main():
    html_text = _fetch(HN_URL)
    df_raw = q2_parse_items(html_text)
    df = q2_clean(df_raw)
    top15 = q2_sort_top(df, top=15)

    # Save CSV with safe encoding
    top15.to_csv("data_q2.csv", index=False, encoding="utf-8-sig")

    # Pretty console output
    print("\nTop 15 Hacker News Stories:\n")
    for i, row in top15.iterrows():
        print(f"{i+1:>2}. ({row['points']} pts, {row['comments']} comments)")
        print(f"    {row['title']}")
        print(f"    Link: {row['link']}\n")

if __name__ == "__main__":
    main()



Top 15 Hacker News Stories:

 1. (894 pts, 353 comments)
    You should write an agent
    Link: https://fly.io/blog/everyone-write-an-agent/

 2. (564 pts, 128 comments)
    Leaving Meta and PyTorch
    Link: https://soumith.ch/blog/2025-11-06-leaving-meta-and-pytorch.md.html

 3. (561 pts, 389 comments)
    Two billion email addresses were exposed
    Link: https://www.troyhunt.com/2-billion-email-addresses-were-exposed-and-we-indexed-them-all-in-have-i-been-pwned/

 4. (513 pts, 209 comments)
    Show HN: I scraped 3B Goodreads reviews to train a better recommendation model
    Link: https://book.sv

 5. (455 pts, 157 comments)
    A Fond Farewell
    Link: https://www.farmersalmanac.com/fond-farewell-from-farmers-almanac

 6. (430 pts, 136 comments)
    Game design is simple
    Link: https://www.raphkoster.com/2025/11/03/game-design-is-simple-actually/

 7. (402 pts, 294 comments)
    Meta projected 10% of 2024 revenue came from scams
    Link: https://sherwood.news/tech/meta-pro